<a href="https://colab.research.google.com/github/rjanow/Masterarbeit/blob/main/SOLYS_to_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SOLYS_to_CSV

Hier werden die Sun-Tracker Daten in eine CSV umgewandelt:

In [1]:
import os
import pandas as pd
import matplotlib.pyplot  as plt

from google.colab import drive

In [2]:
# Pfad zum Ordner der Drive in dem die Messdaten gespeichert sind
folder_path = f'/content/drive/My Drive/Colab_Notebooks/SOLYS_Messdaten'

In [3]:
# Funktion zum Laden aller Dateien in einem Google Drive-Ordner
def load_data_from_drive_folder(folder_path, header_lines=7, encoding = 'cp1252'):

    # Liste, um alle DataFrames zu speichern
    dfs = []

    # Durchlaufen alle Dateien im angegebenen Ordner
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        # Überprüfe, ob es eine Datei ist und lade sie
        if os.path.isfile(file_path):
            df = pd.read_csv(file_path, skiprows=header_lines, sep = '\t', encoding=encoding)
            dfs.append(df)

    # Kombinieren aller DataFrames
    combined_df = pd.concat(dfs, ignore_index=True)

    return combined_df

In [4]:
# Aufruf der Funktion zum Einlesen der Messdaten
df = load_data_from_drive_folder(folder_path)

## Löschen und umbenennen der Spalten:

In [5]:
# Löschen der nicht benötigten Splaten
df_Solys = df.drop(columns = ['Unit', 'Unnamed: 10', ' °C', 'V', ' °C.1', 'Unnamed: 9'], axis=1)

In [6]:
df_Solys

,Unnamed: 1,W/m²,W/m².1,W/m².2,W/m².3,°C.2
0,27.04.2023 06:31:20,"300,0","115,7","311,3","564,5","11,6"
1,27.04.2023 06:31:30,"300,8","116,0","312,1","565,1","11,2"
2,27.04.2023 06:31:40,"301,5","116,2","312,7","565,7","11,1"
3,27.04.2023 06:31:50,"302,2","116,5","313,5","566,1","11,2"
4,27.04.2023 06:32:00,"303,0","116,7","314,2","566,7","11,0"
...,...,...,...,...,...,...
4324954,12.08.2022 12:56:30,"793,8","72,5","784,2","935,0","32,2"
4324955,12.08.2022 12:56:40,"793,5","72,5","783,5","934,9","32,3"
4324956,12.08.2022 12:56:50,"793,8","72,4","784,0","935,6","32,3"
4324957,12.08.2022 12:57:00,"794,1","72,3","784,0","936,2","32,2"


In [7]:
# Umbenennen der Splatennamen
df_Solys.columns = ['Datetime', 'Glo', 'Dif', 'Glo_SPLite', 'Dir', 'Temp']

In [8]:
# Datetime-Spalte als Index setzen
df_Solys.set_index('Datetime', inplace = True)

In [9]:
df_Solys.index = pd.to_datetime(df_Solys.index, format='%d.%m.%Y %H:%M:%S')

In [10]:
df_Solys

,Glo,Dif,Glo_SPLite,Dir,Temp
Datetime,,,,,
2023-04-27 06:31:20,"300,0","115,7","311,3","564,5","11,6"
2023-04-27 06:31:30,"300,8","116,0","312,1","565,1","11,2"
2023-04-27 06:31:40,"301,5","116,2","312,7","565,7","11,1"
2023-04-27 06:31:50,"302,2","116,5","313,5","566,1","11,2"
2023-04-27 06:32:00,"303,0","116,7","314,2","566,7","11,0"
...,...,...,...,...,...
2022-08-12 12:56:30,"793,8","72,5","784,2","935,0","32,2"
2022-08-12 12:56:40,"793,5","72,5","783,5","934,9","32,3"
2022-08-12 12:56:50,"793,8","72,4","784,0","935,6","32,3"


## Messungen in die richtige Reihenfolge bringen:

In [11]:
# Sotieren des Index in chronologischer Reihenfolge
df_Solys.sort_index(ascending=True, inplace = True)

In [12]:
# Prüfen, ob die Messdaten in chronologischer Reihenfolge sind
chronologisch = df_Solys.index.is_monotonic_increasing
print(f"Der Index ist in chronologischer Reihenfolge: {chronologisch}")

Der Index ist in chronologischer Reihenfolge: True


In [23]:
for column in df_Solys.columns:
    try:
        df_Solys[column] = df_Solys[column].str.replace(',', '.').astype(float)
    except ValueError:
        print(f"Spalte {column} konnte nicht in float umgewandelt werden.")

In [24]:
df_Solys.astype(float)

,Glo,Dif,Glo_SPLite,Dir,Temp
Datetime,,,,,
2022-04-25 08:07:30,243.9,244.9,257.1,2.4,10.7
2022-04-25 08:07:40,245.8,246.9,259.3,2.4,10.8
2022-04-25 08:07:50,247.8,248.8,261.3,2.4,10.8
2022-04-25 08:08:00,249.1,250.0,262.4,2.4,10.8
2022-04-25 08:08:10,249.5,250.3,262.7,2.4,10.9
...,...,...,...,...,...
2023-11-09 08:18:50,14.1,14.7,15.2,1.5,8.8
2023-11-09 08:19:00,14.3,14.9,15.4,1.5,8.8
2023-11-09 08:19:10,14.5,15.0,15.6,1.5,8.8


## Prüfen, ob die Daten zusammen hängen:

In [14]:
round(df_Solys.describe(), 2).T

,count,unique,top,freq
Glo,4324959,12133,"0,0",2021462
Dif,4324959,6513,"0,0",1900498
Glo_SPLite,4324959,11860,"0,0",1550908
Dir,4324959,10155,"1,4",409326
Temp,4324959,570,"17,6",23341


In [25]:
def pruefe_zeitlichen_abstand(df, min_abstand):

  ergebnisse = []

  for datum, gruppe in df.groupby(df.index.date):
        gruppe = gruppe.sort_index()
        abstaende = gruppe.index.to_series().diff().dropna()
        erfuellt = all(abstaende >= min_abstand)
        ergebnisse.append({'Datum': datum, 'Erfuellt': erfuellt})

  ergebnis_df = pd.DataFrame(ergebnisse)
  ergebnis_df['Datum'] = pd.to_datetime(ergebnis_df['Datum'])

  return ergebnis_df

In [26]:
# Aufruf der Funktion um zu prüfen, ob die Daten zusammenhängend sind,
ergebnis_df = pruefe_zeitlichen_abstand(df_Solys, pd.Timedelta('10S'))
print(len(pd.unique(df_Solys.index.date)), len(ergebnis_df[ergebnis_df['Erfuellt']]))

504 499


## Daten reduzieren:

Daten reduzieren damit die Auflösung mit der des Spektrometers überein stimmt:

In [28]:
# Durchschnittswerte pro Tag berechnen
df_Solys_mean = df_Solys.resample('2T').mean()

In [19]:
df_Solys_mean

""
Datetime
2022-04-25 08:06:00
2022-04-25 08:08:00
2022-04-25 08:10:00
2022-04-25 08:12:00
2022-04-25 08:14:00
...
2023-11-09 08:10:00
2023-11-09 08:12:00
2023-11-09 08:14:00


## Abspeichern der Messdaten als CSV in der Google-Drive